In [1]:
import pandas as pd
import pysam
import numpy as np
import sys
from collections import defaultdict
import time
start_time = time.time()

bamfile ='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/softRD_pileup/CD4allrange_NR_1000000_insilmix41_sorted'

#####sm must be hypo###

sm='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/softRD_pileup/BL14_all_matrixCin_nr0.4_imputed_rowmean.txt_bg_intesectedwith_BL14ateast.2DMRs.txt_othermean.txt'

mapping_quality=40
base_quality=20


deltagreaterforpositive=0  ##############>0

deltasmallerfornegative=0 ############<0



In [2]:
smdf=pd.read_csv(sm,sep='\t',index_col=['chrom','start'])

smdf.head()
   

CD4-others  CD8-others  nB-others  NK-others  Mn-others  \
chrom start                                                              
chr1  1221863    0.107538   -0.014513   0.115436  -0.782718   0.084564   
      1221897    0.099756    0.010731   0.105500  -0.790141   0.085397   
      1222070    0.110564    0.038410   0.103026  -0.860103   0.114154   
      1222077    0.090962    0.007321   0.157013  -0.859962   0.146244   
      1222089    0.107667    0.025103   0.093667  -0.853667   0.101205   

               mNeu-others  m8-others  DC-others  Eo-others  Tr-others  \
chrom start                                                              
chr1  1221863     0.119026   0.114359   0.028205   0.031974   0.102513   
      1221897     0.092218   0.069423   0.037295  -0.010269   0.078577   
      1222070     0.078256   0.063179   0.066051  -0.020821   0.089564   
      1222077     0.069423   0.105859   0.041423  -0.007218   0.085936   
      1222089     0.030846   0.085410   0.010385   0.000872   0.131179   

               m4-others  Mg-others  Er-others  mB-others  
chrom start                                                
chr1  1221863   0.045974   0.000205   0.041128   0.006308  
      1221897   0.028500   0.090423   0.066731   0.035859  
      1222070  -0.000897   0.083103   0.058333   0.077179  
      1222077  -0.022295   0.075705   0.047705   0.061885  
      1222089   0.080564   0.080026   0.049333   0.057410

In [3]:
scorecolumns=smdf.columns
scoredict=defaultdict(list)

In [4]:
hypoCpGdict=defaultdict(list)
lenhypoCpGdict=defaultdict(list)
hyperCpGdict=defaultdict(list)
lenhyperCpGdict=defaultdict(list)

In [5]:
def populate_score_matrix(aRead,direction):
    Refstart=aRead.reference_start
    

    Refend=aRead.reference_end-1
    

    Refchrom=aRead.reference_name
    

    
    
    
    corressm=smdf.loc[(Refchrom,Refstart):(Refchrom,Refend)]
    
    
    
    if corressm.shape[0]==0:
        return
    
    Readname=aRead.query_name
    
    if Readname=='m4.HWI-ST537:241:C73HGACXX:2:1105:2196:59768':
        print(corressm)
        
        print(aRead.is_secondary)
        print(aRead.is_supplementary)
        print(aRead.get_aligned_pairs(with_seq=True) )
        sys.exit(1)
        
    readbase_qualities=aRead.query_qualities
    readbases=aRead.query_sequence
    refpositions=aRead.get_reference_positions(full_length=True)
    
    len_readbases=len(readbases)
    
    if len(readbase_qualities)!=len_readbases:
        print('len_readbase_qualities!=len_readbases')
        sys.exit(1)

    if len(refpositions)!=len_readbases:
        print("something wrong")
        print(aRead)
        print(Refend)
        print(Refstart)
        print(len(readbases))
        print(aRead.get_reference_positions(full_length=True))
        
        print(aRead.get_aligned_pairs(with_seq=True))
        
        sys.exit(1)
        
    for cindex, crow in corressm.iterrows():
        
    
        try:
            baseposindex=refpositions.index(cindex[1]+direction)
        except:
            continue
      
        

        
        
        readbase=readbases[baseposindex]
        
        
        
        if Readname=='mNeu.HWI-ST700660_130:3:2212:4886:72504#20' and cindex[1]==1222104:
            print(readbase)
            print(direction)
            print(refpositions)
            
            print(cindex[1])
            print(baseposindex)
            
            print(readbases)

            print(aRead.get_aligned_pairs(with_seq=True) )
            sys.exit(1)
        
        
        if readbase==None:
            continue
        
        
        if readbase_qualities[baseposindex]<base_quality:
            continue
        
        readbase=readbase.upper()
        
        
        if Readname=='mNeu.HWI-ST700660_130:3:2212:4886:72504#20' and cindex[1]==1222104:
            print(readbase)
            print(direction)
            print(refpositions)
            
            print(cindex[1])
            print(baseposindex)
            
            print(readbases)

            print(aRead.get_aligned_pairs(with_seq=True) )
            sys.exit(1)
        
        accpeted=0
        
        
        if direction==0:
            if readbase=='T':
                accpeted=-1
                
                if len(hypoCpGdict[Readname])==0:
                    hypoCpGdict[Readname]=[[Refchrom+":"+str(cindex[1])]]
             
                
                elif len(hypoCpGdict[Readname])==1:
                    
                    hypoCpGdict[Readname]=[hypoCpGdict[Readname][0]+[Refchrom+":"+str(cindex[1])]]
                    
                else:
                    print('prob in hypoCpGdict')
                    sys.exit(1)
                    
                
                if len(lenhypoCpGdict[Readname])==0:
                    lenhypoCpGdict[Readname]=[1]
                elif len(lenhypoCpGdict[Readname])==1:
                    lenhypoCpGdict[Readname]=[lenhypoCpGdict[Readname][0]+1]
                else:
                    print('prob in lenhypoCpGdict')
                    sys.exit(1)
                
             
            elif readbase=='C':
                if len(hyperCpGdict[Readname])==0:
                    hyperCpGdict[Readname]=[[Refchrom+":"+str(cindex[1])]]
                
                elif len(hyperCpGdict[Readname])==1:
                    hyperCpGdict[Readname]=[hyperCpGdict[Readname][0]+[Refchrom+":"+str(cindex[1])]]
                    
                else:
                    
                    print('prob in hyperCpGdict')
                    sys.exit(1)
                
                if len(lenhyperCpGdict[Readname])==0:
                    lenhyperCpGdict[Readname]=[1]
                elif len(lenhyperCpGdict[Readname])==1:
                    
                    lenhyperCpGdict[Readname]=[lenhyperCpGdict[Readname][0]+1]
                else:
                    print('prob in lenhyperCpGdict')
                    sys.exit(1)
                
            #    accpeted=1
            
                
        elif direction==1:
            if readbase=='A':
                accpeted=-1
                
                if len(hypoCpGdict[Readname])==0:
                    hypoCpGdict[Readname]=[[Refchrom+":"+str(cindex[1])]]
                
                elif len(hypoCpGdict[Readname])==1:
                    hypoCpGdict[Readname]=[hypoCpGdict[Readname][0]+[Refchrom+":"+str(cindex[1])]]
                    
                else:
                    print('prob in hypoCpGdict')
                    sys.exit(1)
                    
                                                                                    
                
                if len(lenhypoCpGdict[Readname])==0:
                    lenhypoCpGdict[Readname]=[1]
                elif len(lenhypoCpGdict[Readname])==1:
                    
                    lenhypoCpGdict[Readname]=[lenhypoCpGdict[Readname][0]+1]
                else:
                    print('prob in lenhypoCpGdict')
                    sys.exit(1)
            elif readbase=='G':
                if len(hyperCpGdict[Readname])==0:
                    hyperCpGdict[Readname]=[[Refchrom+":"+str(cindex[1])]]
                
                elif len(hyperCpGdict[Readname])==1:
                    hyperCpGdict[Readname]=[hyperCpGdict[Readname]+[Refchrom+":"+str(cindex[1])]]
                    
                else:
                    
                    print('prob in hyperCpGdict')
                    sys.exit(1)
                
                if len(lenhyperCpGdict[Readname])==0:
                    lenhyperCpGdict[Readname]=[1]
                elif len(lenhyperCpGdict[Readname])==1:
                    
                    lenhyperCpGdict[Readname]=[lenhyperCpGdict[Readname][0]+1]
                else:
                    print('prob in lenhyperCpGdict')
                    sys.exit(1)
                
            #    accpeted=1
        
        if accpeted!=0 :
            
            smrow=crow.to_numpy()
            
            
            if len(scoredict[Readname])==0:
                    scoredict[Readname]=list(accpeted*smrow)
            else:
                scoredict[Readname]=list(np.array(scoredict[Readname])+(accpeted*smrow))
            
            
                
                
            

            
        
      
    
    
    
    
  
    
    
 
        
    

In [6]:
bam=pysam.AlignmentFile(bamfile, 'rb')

for read in bam.fetch():
    if read.mapping_quality < mapping_quality:
        continue
    if read.is_duplicate == True:
        continue
    
    offset = 0

    

    if read.has_tag('YD'):
        yd = read.get_tag("YD")
        if yd == "r":
            offset = 1

    elif read.has_tag('XB'):
        xb=read.get_tag("XB")

        if xb=='G':
            offset=1
        elif xb=='C':
            pass
        else:
            print("unknown xb value. Exiting....")
            print(read.query_name)

            sys.exit(1)

    else:
        print("no tag found. Exiting")
        print(read.query_name)
        sys.exit(1)
        
        
    
    


    populate_score_matrix(read,offset)
    

               CD4-others  CD8-others  nB-others  NK-others  Mn-others  \
chrom start                                                              
chr1  1222070    0.110564    0.038410   0.103026  -0.860103   0.114154   
      1222077    0.090962    0.007321   0.157013  -0.859962   0.146244   

               mNeu-others  m8-others  DC-others  Eo-others  Tr-others  \
chrom start                                                              
chr1  1222070     0.078256   0.063179   0.066051  -0.020821   0.089564   
      1222077     0.069423   0.105859   0.041423  -0.007218   0.085936   

               m4-others  Mg-others  Er-others  mB-others  
chrom start                                                
chr1  1222070  -0.000897   0.083103   0.058333   0.077179  
      1222077  -0.022295   0.075705   0.047705   0.061885  
False
False
[(0, 1221983, 'T'), (1, 1221984, 'c'), (2, 1221985, 'T'), (3, 1221986, 'G'), (4, 1221987, 'G'), (5, 1221988, 'T'), (6, 1221989, 'G'), (7, 1221990, 'A'), (

SystemExit: 1

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
scoredf=pd.DataFrame.from_dict(scoredict, orient='index',columns=scorecolumns)
scoredf.head()

In [ ]:
#lenhypoCpGdict

In [ ]:
hypoCpGdf=pd.DataFrame.from_dict(hypoCpGdict, orient='index',columns=['hypoCpG'])
lenhypoCpGdf=pd.DataFrame.from_dict(lenhypoCpGdict, orient='index',columns=['LENhypoCpG'])
hyperCpGdf=pd.DataFrame.from_dict(hyperCpGdict, orient='index',columns=['hyperCpG'])
lenhyperCpGdf=pd.DataFrame.from_dict(lenhyperCpGdict,orient='index',columns=['LENhyperCpG'])

mergedcpgdf=pd.concat([hypoCpGdf,hyperCpGdf])
mergedcpgdf=mergedcpgdf.fillna('NotFound')

mergedLENcpgdf=pd.concat([lenhypoCpGdf,lenhyperCpGdf])
mergedLENcpgdf=mergedLENcpgdf.fillna(0)


if (mergedLENcpgdf.shape[0]==mergedcpgdf.shape[0]):
    cpginfomerged=pd.concat([mergedcpgdf,mergedLENcpgdf],axis=1)
    
    outdf=pd.concat([scoredf,lenhypoCpGdf,hypoCpGdf],axis=1)
else:
    
    print('problem in merging')
    sys.exit(1)

In [ ]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

In [ ]:
cpginfomerged.to_csv(bamfile+"_scored_nohyper_cpgoinfomerged.txt",sep="\t")
outdf.to_csv(bamfile+"_scored_nohyper.txt",sep="\t")

In [ ]:
ctpropdict= defaultdict(list)

for score in scorecolumns:
    

    
    posscore=scoredf.loc[scoredf[score]>deltagreaterforpositive,score]
    
    negscore=scoredf.loc[scoredf[score]<deltasmallerfornegative,score]
    
    
    poslen=len(posscore)
    neglen=len(negscore)
    
    posscoreSUM=sum(posscore)
    
    
    negscoreSUM=sum(negscore)
    
    
    
    ct_posnegscore=posscoreSUM #/(posscoreSUM+abs(negscoreSUM))
    
    
    ct_fragscore=poslen #/(poslen+neglen)
    
    
    
    
    
    ctname=score.replace('-others','')
    
    ctpropdict[ctname+"_posnegscore"].append(ct_posnegscore)

    ctpropdict[ctname+"_fragscore"].append(ct_fragscore)
    
    
    

In [ ]:
#print(posscore)

In [ ]:
ctpropdf=pd.DataFrame.from_dict(ctpropdict)
ctpropdf.to_csv(bamfile+"_softRD_nohyper_result.txt",sep="\t",index=False)

In [ ]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)